In [1]:
import re
import pandas as pd
import numpy as np
import MySQLdb
from scipy.ndimage.interpolation import shift
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix, hstack
from sklearn.model_selection import train_test_split
from datetime import datetime
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import NuSVC, SVC
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn import preprocessing
from sklearn.model_selection import cross_val_predict, cross_val_score, StratifiedKFold, KFold
from keras.models import Sequential
from keras import optimizers
from keras.layers import Dense, Activation, Dropout, LSTM, Merge, Input, Embedding
from keras.layers import Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras.utils import np_utils

Using TensorFlow backend.


In [2]:
def getdata(news_class):
    conn = MySQLdb.connect(host='54.65.19.253', port=3306, user='news', passwd='newsnews', db='news')
    news = pd.read_csv('news_title.csv')
    news = news.loc[news['class'].isin(news_class)]
    news = news.drop(['date', 'class'], axis=1)
    news = news.groupby('timestamp', as_index=False).agg(lambda name: ' '.join(name))
    gbp = pd.read_sql('SELECT gbp.timestamp, gbp \
                FROM gbp \
                WHERE gbp.timestamp >= 1262275200;', conn)
    news = pd.merge(news, gbp, on='timestamp', how='inner')
    return news

### "world","politics","sport","football","culture","business","lifeandstyle",
### "fashion","environment","technology","travel

In [3]:
data = pd.read_csv('newstitle_gbp_xau.csv')
# news_class = ['world','business']
# data = getdata(news_class)
data.shape

(3457, 4)

In [4]:
data['gbp_gradient'] = np.gradient(data.as_matrix(columns=['gbp']).reshape(-1))

In [5]:
def get_shift(data, shift_offset=2):
    g = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
    for i in range(1, shift_offset+1):
        data['gbp_gradient_p_'+str(i)] = shift(g, i, cval=0)

In [6]:
get_shift(data, shift_offset=2)

In [7]:
data.head()

,timestamp,title,gbp,xau,gbp_gradient,gbp_gradient_p_1,gbp_gradient_p_2
0,1167580800,Italy puts froth back into cappuccino Patients...,1.959094,324.767450,0.014348,0.000000,0.000000
1,1167667200,Pork soup handouts not racist - French judge D...,1.973442,324.524002,-0.004827,0.014348,0.000000
2,1167753600,Iraqi PM rules out second term Zimbabwe poised...,1.949440,322.762609,-0.014576,-0.004827,0.014348
3,1167840000,Millions get the message as texting hits a fes...,1.944289,322.798264,-0.009760,-0.014576,-0.004827
4,1167926400,Three of Pinochet family cleared News in Brief...,1.929921,314.234051,-0.006893,-0.009760,-0.014576


In [8]:
def binary(Y):
    Y[np.where(Y > 0)] = 1
    Y[np.where(Y <= 0)] = 0
    Y = Y.astype('int64')
    f = np.bincount(Y)
    print(f/np.sum(f))
    return Y

In [9]:
Y = data.as_matrix(columns=['gbp_gradient']).reshape(-1)
Y = binary(Y)
num_classes = 2
#Y = np_utils.to_categorical(Y, num_classes)

[ 0.48654903  0.51345097]


In [10]:
X_p = data.as_matrix(columns=['gbp_gradient_p_1', 'gbp_gradient_p_2'])
X_p[:,0] = binary(X_p[:,0])
X_p[:,1] = binary(X_p[:,1])

[ 0.48625976  0.51374024]
[ 0.48654903  0.51345097]


In [11]:
feature_name = 'title'
vectorizer = TfidfVectorizer(min_df=50, ngram_range=(1, 4))
X = vectorizer.fit_transform(data[feature_name].tolist())
X

<3457x652 sparse matrix of type '<class 'numpy.float64'>'
	with 88673 stored elements in Compressed Sparse Row format>

In [12]:
with_news = True
num_epochs = 34

In [13]:
if with_news:
    X = hstack([X, X.power(2), X.power(3), X.power(4), X_p])
else:
    X = X_p

In [14]:
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size=0.8)
# print(X_train.shape)
# print(X_test.shape)
# if with_news:
#     X_train = X_train.toarray()
#     X_test = X_test.toarray()

In [15]:
def create_model():
    model = Sequential()
    model.add(Dense(10, activation='tanh', input_shape=(X.shape[1],), bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='tanh', bias_initializer='RandomNormal'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='RMSprop',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    #model.summary()
    return model

In [16]:
model = create_model()

In [17]:
# history = model.fit(X_train, y_train, 
#                     epochs=32,
#                     validation_data = (X_test, y_test),
#                     batch_size=1000)

In [18]:
# pred = np.argmax(model.predict(X_test), axis=1)
# true = np.argmax(y_test, axis=1)
# print(classification_report(true, pred, target_names=['down','up']))

In [19]:
# cv = 5
# recall = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='recall')
# precision = cross_val_score(model, X, Y, cv=cv, n_jobs = -1, scoring='precision')
# print('\nprecision:',np.mean(precision))
# print('recall:', np.mean(recall))
# print('acc: %.4f'%(np.mean(cross_val_score(model, X, Y, cv=cv, n_jobs = -1))))

In [20]:
def show_result(X_test, y_test):
    target_names = ['decline','up']
    pred = np.argmax(model.predict(X_test), axis=1)
    #print(confusion_matrix(y_test, pred, labels=[0,1]))
    mat = confusion_matrix(y_test, pred, labels=[0,1])
    #print(classification_report(y_test, pred, target_names=target_names))
    return (mat[0,0]+mat[1,1])/np.sum(mat)

In [21]:
skf = StratifiedKFold(n_splits=5, shuffle=True)
acc = []
for i in range(5): #做數次隨機StratifiedKFold,求出更平均的數值
    for train_idx, test_idx in skf.split(X, Y):
        print ("Running Fold")
        model = create_model()
        if with_news:
            x = X.toarray()
        else:
            x = X
        y = np_utils.to_categorical(Y, num_classes)
        history = model.fit(x[train_idx], y[train_idx], 
                        epochs=num_epochs,
                        batch_size=4000,
                        verbose = 0)
        acc.append(show_result(x[test_idx], Y[test_idx]))
    print(i)
print('acc:%.4f'%(np.mean(acc)))

Running Fold
Running Fold
Running Fold
Running Fold
Running Fold
0
Running Fold
Running Fold
Running Fold
Running Fold
Running Fold
1
Running Fold
Running Fold
Running Fold
Running Fold
Running Fold
2
Running Fold
Running Fold
Running Fold
Running Fold
Running Fold
3
Running Fold
Running Fold
Running Fold
Running Fold
Running Fold
4
acc:0.6098


* acc:0.6300 with world
* acc:0.6280 with ['world','technology']
* acc:0.6253 with ['world','business']
* acc:0.6238 with technology
* acc:0.6233 with  ['world', 'business', 'technology']
* acc:0.6225 with business
* acc:0.6221 with environment 
* acc:0.6209 with football
* acc:0.6205 with lifeandstyle
* acc:0.6178 with sport
* acc:0.6125 with politics
* acc:0.6051 with ['world','business', 'politics']
* acc:0.5997 without any thing

In [22]:
#acc:0.6253